In [ ]:
# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
from transformers import AutoTokenizer, Qwen3ForCausalLM

model = Qwen3ForCausalLM.from_pretrained("Qwen/Qwen3-0.6B",cache_dir="./.cache",device_map="auto", torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B",cache_dir="./.cache",device_map="auto", torch_dtype="auto")

prompt = "Hey, are you conscious? Can you talk to me?"
### (2) 챗 템플릿
chat = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user",   "content": f"<image>"+prompt},
]
inputs = tokenizer.apply_chat_template(
            chat,
            add_generation_prompt=True,     # 마지막에 <|im_start|>assistant 삽입
            enable_thinking=False,
            return_tensors="pt"             # input_ids, attention_mask 직접 반환
        )
print(inputs)

inputs = inputs.to(model.device)
# Generate
generate_ids = model.generate(inputs)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)[0]

In [1]:
from transformers import (
    AutoProcessor,
    AutoTokenizer,
    AutoModel,
    AutoConfig,
    Gemma3ForConditionalGeneration,
    Qwen2_5_VLForConditionalGeneration, 
    OPTForCausalLM,
    VisionEncoderDecoderConfig,
    VisionEncoderDecoderModel
)
# from src.models.config import VisionLanguageConfig

/Users/seungwoo/anaconda3/envs/sur/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### QWEN2.5VL


In [7]:
model_name = "Qwen/Qwen2.5-VL-3B-Instruct"
model_config = AutoConfig.from_pretrained(model_name, trust_remote_code=True)
processor = AutoProcessor.from_pretrained(model_name, trust_remote_code=True)
print(model_config)

Qwen2_5_VLConfig {
  "architectures": [
    "Qwen2_5_VLForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "image_token_id": 151655,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 128000,
  "max_window_layers": 70,
  "model_type": "qwen2_5_vl",
  "num_attention_heads": 16,
  "num_hidden_layers": 36,
  "num_key_value_heads": 2,
  "rms_norm_eps": 1e-06,
  "rope_scaling": {
    "mrope_section": [
      16,
      24,
      24
    ],
    "rope_type": "default",
    "type": "default"
  },
  "rope_theta": 1000000.0,
  "sliding_window": 32768,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.51.3",
  "use_cache": true,
  "use_sliding_window": false,
  "video_token_id": 151656,
  "vision_config": {
    "depth": 32,
    "fullatt_block_indexes": [
      7,
      15,
      23,
      31
    ],
    "hidden_act": "

In [6]:
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    model_name,
    # config=model_config,
    cache_dir="./.cache",
)
print(model)

Loading checkpoint shards: 100%|██████████| 2/2 [00:15<00:00,  7.95s/it]


Qwen2_5_VLForConditionalGeneration(
  (visual): Qwen2_5_VisionTransformerPretrainedModel(
    (patch_embed): Qwen2_5_VisionPatchEmbed(
      (proj): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
    )
    (rotary_pos_emb): Qwen2_5_VisionRotaryEmbedding()
    (blocks): ModuleList(
      (0-31): 32 x Qwen2_5_VLVisionBlock(
        (norm1): Qwen2RMSNorm((1280,), eps=1e-06)
        (norm2): Qwen2RMSNorm((1280,), eps=1e-06)
        (attn): Qwen2_5_VLVisionSdpaAttention(
          (qkv): Linear(in_features=1280, out_features=3840, bias=True)
          (proj): Linear(in_features=1280, out_features=1280, bias=True)
        )
        (mlp): Qwen2_5_VLMLP(
          (gate_proj): Linear(in_features=1280, out_features=3420, bias=True)
          (up_proj): Linear(in_features=1280, out_features=3420, bias=True)
          (down_proj): Linear(in_features=3420, out_features=1280, bias=True)
          (act_fn): SiLU()
        )
      )
    )
    (merger): Qwen2_5_VLPatchMerger

In [4]:
model = model.model
print(model)

Qwen2_5_VLModel(
  (embed_tokens): Embedding(151936, 2048)
  (layers): ModuleList(
    (0-35): 36 x Qwen2_5_VLDecoderLayer(
      (self_attn): Qwen2_5_VLSdpaAttention(
        (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
        (k_proj): Linear(in_features=2048, out_features=256, bias=True)
        (v_proj): Linear(in_features=2048, out_features=256, bias=True)
        (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        (rotary_emb): Qwen2_5_VLRotaryEmbedding()
      )
      (mlp): Qwen2MLP(
        (gate_proj): Linear(in_features=2048, out_features=11008, bias=False)
        (up_proj): Linear(in_features=2048, out_features=11008, bias=False)
        (down_proj): Linear(in_features=11008, out_features=2048, bias=False)
        (act_fn): SiLU()
      )
      (input_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
      (post_attention_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
    )
  )
  (norm): Qwen2RMSNorm((2048,), eps=1e-06)
  (rotary_emb): Q

### GEMMA3

In [2]:
model_name = "google/gemma-3-4b-it"
model_config = AutoConfig.from_pretrained(model_name, trust_remote_code=True)
processor = AutoProcessor.from_pretrained(model_name, trust_remote_code=True)
print(model_config)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Gemma3Config {
  "architectures": [
    "Gemma3ForConditionalGeneration"
  ],
  "boi_token_index": 255999,
  "eoi_token_index": 256000,
  "eos_token_id": [
    1,
    106
  ],
  "image_token_index": 262144,
  "initializer_range": 0.02,
  "mm_tokens_per_image": 256,
  "model_type": "gemma3",
  "text_config": {
    "attention_bias": false,
    "attention_dropout": 0.0,
    "attn_logit_softcapping": null,
    "cache_implementation": "hybrid",
    "final_logit_softcapping": null,
    "head_dim": 256,
    "hidden_activation": "gelu_pytorch_tanh",
    "hidden_size": 2560,
    "initializer_range": 0.02,
    "intermediate_size": 10240,
    "max_position_embeddings": 131072,
    "model_type": "gemma3_text",
    "num_attention_heads": 8,
    "num_hidden_layers": 34,
    "num_key_value_heads": 4,
    "query_pre_attn_scalar": 256,
    "rms_norm_eps": 1e-06,
    "rope_local_base_freq": 10000.0,
    "rope_scaling": {
      "factor": 8.0,
      "rope_type": "linear"
    },
    "rope_theta": 1000000.0

In [ ]:
model = Gemma3ForConditionalGeneration.from_pretrained(
    model_name,
    # config=model_config,
    cache_dir="./.cache",
)
print(model)
model = model.language_model
print(model)

In [ ]:
import transformers

MODEL_TYPE_MAPPING = {
    'google/gemma-3-4b-it': transformers.Gemma3ForConditionalGeneration,
    'Qwen/Qwen2.5-VL-3B-Instruct': transformers.Qwen2_5_VLForConditionalGeneration,
}

def get_language_model_class(config):
    """
    매핑 테이블을 활용한 모델 클래스 선택
    """
    model_type = config.get('model_type', '')
    
    # 정확한 매치 먼저 확인
    if model_type in MODEL_TYPE_MAPPING:
        return MODEL_TYPE_MAPPING[model_type].language_model
    
    